# Send Jobs from your Computer

<img src="../../imgs/Upload_Run_Download.png" alt="Procedure" height="250" title="Procedure for the Python Client"/>

In [6]:
import sys
sys.path.append('../..')
from src.VipSession import VipSession
from pathlib import *

## Initiate your Session 

*API key is provided in your VIP account settings*

In [7]:
VipSession.init(api_key="VIP_API_KEY"); # Paste your VIP API key here
# N.B.: API key can be stored in an environment variable, in a file or written as litteral string


----------------------------------
| You are communicating with VIP |
----------------------------------



In [8]:
# Instantiate a Session with a name
my_session = VipSession("demo-single-job")


=== SESSION 'demo-single-job' ===

Output directory: vip_outputs/demo-single-job



## Upload your dataset on VIP

Upload a single folder containing the full dataset

In [9]:
input_dir = "data/quest_1job"
! tree {input_dir}

data/quest_1job
├── basis.zip
├── parameters.txt
└── signal.mrui

0 directories, 3 files


In [10]:
my_session.upload_inputs(input_dir);


=== UPLOAD INPUTS ===

Input Directory: 'data/quest_1job' --> checked

Uploading the dataset on VIP

-----------------------------
Cloning: data/quest_1job ... Already on VIP.
-----------------------------
Everything is on VIP.

>> Session was saved in: vip_outputs/demo-single-job/session_data.json



*The session is automatically backed up in its output directory* 

In [11]:
! tree {Path(my_session.output_dir).parent} 

vip_outputs
└── demo-single-job
    └── session_data.json

1 directory, 1 file


In [12]:
! head {Path(my_session.output_dir) / "session_data.json"} 

{
    "session_name": "demo-single-job",
    "pipeline_id": null,
    "local_input_dir": "data/quest_1job",
    "local_output_dir": "vip_outputs/demo-single-job",
    "vip_input_dir": "/vip/Home/API/demo-single-job/INPUTS",
    "vip_output_dir": "/vip/Home/API/demo-single-job/OUTPUTS",
    "input_settings": null,
    "workflows": {}
}

## Launch the pipeline

### Pipeline Identifier

Show available pipelines on VIP

In [13]:
VipSession.show_pipeline("cquest")


Available pipelines
-------------------
CQUEST/0.1.1
CQUEST_fuzzy/0.1
-------------------


App/Version

In [14]:
pipeline_id = "CQUEST/0.1.1"

### Input Settings

Show the pipeline description to know which inputs are required by the app

In [15]:
VipSession.show_pipeline(pipeline_id)

-----------------------------
name: CQUEST | version: 0.1.1
-----------------------------
pipeline_id: CQUEST/0.1.1
-----------------------------
input_settings:
 - 'zipped_folder': [File] Archive containing the files listed in the parameter file
 - 'parameter_file': [File] File setting up constraints, options and prior knowledge used in cQuest algorithm
 - 'data_file': [File] File with extension and format of jMRUI containing the signal to quantify
-----------------------------



Provide the inputs

In [16]:
# Inputs can take several formats
input_settings = { 
    "zipped_folder" : "data/quest_1job/basis.zip", # String
    "parameter_file" : Path(input_dir) / "parameters.txt", # PathLib object
    "data_file" : ["data/quest_1job/signal.mrui"] # List of strings / PathLib objects
}

## Run the app

In [17]:
my_session.launch_pipeline(pipeline_id, input_settings);


=== LAUNCH PIPELINE ===

Pipeline ID: 'CQUEST/0.1.1' --> checked
Input Settings --> parsed
Parameter checks
----------------
Pipeline identifier: OK
Output directory: OK
Input settings: OK
----------------

Launching 1 new execution(s) on VIP
-------------------------------------
Execution Name: demo-single-job
Started Workflows:
	workflow-sPobH7, 
-------------------------------------
Done.

>> Session saved



*See progression on https://vip.creatis.insa-lyon.fr/* ...

... or monitor the workflow from this terminal:

In [19]:
my_session.monitor_workflows(refresh_time=2);


=== MONITOR WORKFLOW ===

Updating worflow inventory ... Done.
All executions are currently running on VIP.

-------------------------------------------------------------
The current proccess will wait until all executions are over.
Their progress can be monitored on VIP portal:
	https://vip.creatis.insa-lyon.fr/
-------------------------------------------------------------
All executions are over.
All executions (1) ended with success.

>> Session saved



## Download your Results

In [20]:
my_session.download_outputs();


=== DOWNLOAD OUTPUTS ===

Updating workflow status ... Done.

--------------------------------
[1/1] Outputs from: workflow-sPobH7 | Started on: 2023/06/13 20:08:04 | Status: Finished
	New directory: vip_outputs/demo-single-job/13-06-2023_20:08:05
	[1/1] Downloading file (0.1MB): signal.mrui--parameters.txt.tgz ... Done.
		Extracting archive content ... Done.
	Done for all files.
--------------------------------
Done for all executions.



*If `output_dir` is not specified, results are stored at default location*

In [21]:
! tree {my_session.output_dir}

vip_outputs/demo-single-job
├── 13-06-2023_20:08:05
│   └── signal.mrui--parameters.txt.tgz
│       ├── parameters.txt
│       ├── signal_Correlation.txt
│       ├── signal.mrui
│       ├── signal_quest2.txt
│       ├── signal_quest_back.mrui
│       ├── signal_quest_estim.mrui
│       └── stdout_cquest.txt
└── session_data.json

2 directories, 8 files


# Parallelize your Jobs for Bigger Datasets

*Suppose we have more signals to analyse*

In [22]:
new_dataset = Path("data/quest_2jobs")
! tree {new_dataset}

data/quest_2jobs
├── basis.zip
├── parameters.txt
└── signals
    ├── 001.mrui
    └── 002.mrui

1 directory, 4 files


**_Lists_** of parameters launch **_parallel_** jobs on VIP (here in `data_file`)

In [23]:
# New input parameters for the pipeline
new_settings = {
    "data_file" : list((new_dataset / "signals").iterdir()),
    "parameter_file" : new_dataset / "parameters.txt",
    "zipped_folder" : new_dataset / "basis.zip",
}

In [24]:
# Display the previous settings as strings to see the list of files
import json
print("new_settings =",
    json.dumps(indent=2, obj={
        key: [str(v) for v in value] if isinstance(value, list) else str(value)
            for key, value in new_settings.items()
        }
    )
)

new_settings = {
  "data_file": [
    "data/quest_2jobs/signals/001.mrui",
    "data/quest_2jobs/signals/002.mrui"
  ],
  "parameter_file": "data/quest_2jobs/parameters.txt",
  "zipped_folder": "data/quest_2jobs/basis.zip"
}


Run a new session with the new parameters

In [25]:
new_session = VipSession(
    session_name = "demo-parallel-jobs",
    input_dir = new_dataset,
    pipeline_id = "CQUEST/0.1.1",
    input_settings = new_settings
).run_session();


=== SESSION 'demo-parallel-jobs' ===

Output directory: vip_outputs/demo-parallel-jobs
Pipeline ID: 'CQUEST/0.1.1' --> checked
Input Settings --> parsed
Input Directory: 'data/quest_2jobs' --> checked


=== UPLOAD INPUTS ===

Checking references to the dataset within Input Settings ... 

OK.

Uploading the dataset on VIP

-----------------------------
Cloning: data/quest_2jobs ... (Created on VIP)
	2 files to upload.
	[1/2] Uploading file: basis.zip (0.6MB) ... Done.
	[2/2] Uploading file: parameters.txt (0.0MB) ... Done.
Cloning: data/quest_2jobs/signals ... (Created on VIP)
	2 files to upload.
	[1/2] Uploading file: 001.mrui (0.0MB) ... Done.
	[2/2] Uploading file: 002.mrui (0.0MB) ... Done.
-----------------------------
Everything is on VIP.

>> Session was saved in: vip_outputs/demo-parallel-jobs/session_data.json


=== LAUNCH PIPELINE ===

Parameter checks
----------------
Pipeline identifier: OK
Output directory: Created on VIP
Input settings: OK
----------------

Launching 1 ne

Check the output files

In [26]:
! tree {new_session.output_dir}

vip_outputs/demo-parallel-jobs
├── 13-06-2023_20:10:26
│   ├── 001.mrui--parameters.txt.tgz
│   │   ├── 001_Correlation.txt
│   │   ├── 001.mrui
│   │   ├── 001_quest2.txt
│   │   ├── 001_quest_back.mrui
│   │   ├── 001_quest_estim.mrui
│   │   ├── parameters.txt
│   │   └── stdout_cquest.txt
│   └── 002.mrui--parameters.txt.tgz
│       ├── 002_Correlation.txt
│       ├── 002.mrui
│       ├── 002_quest2.txt
│       ├── 002_quest_back.mrui
│       ├── 002_quest_estim.mrui
│       ├── parameters.txt
│       └── stdout_cquest.txt
└── session_data.json

3 directories, 15 files


# Repeat a Previous Experiment

Resume previous session "demo_single_job" and relaunch it *as is*

In [27]:
VipSession("demo-single-job").run_session(update_files=False, refresh_time=2);


=== SESSION 'demo-single-job' ===

Output directory: vip_outputs/demo-single-job
<< Session restored from its output directory


=== UPLOAD INPUTS ===

Skipped : There are already input data on VIP.

=== LAUNCH PIPELINE ===

Parameter checks
----------------
Pipeline identifier: OK
Output directory: OK
Input settings: OK
----------------

Launching 1 new execution(s) on VIP
-------------------------------------
Execution Name: demo-single-job
Started Workflows:
	workflow-v0T4Ux, 
-------------------------------------
Done.

>> Session saved


=== MONITOR WORKFLOW ===

Updating worflow inventory ... Done.
1 execution(s) ended with success:
	 workflow-sPobH7 , started on: 2023/06/13 20:08:04
1 execution(s) is/are currently running on VIP:
	 workflow-v0T4Ux , started on: 2023/06/13 20:13:11

-------------------------------------------------------------
The current proccess will wait until all executions are over.
Their progress can be monitored on VIP portal:
	https://vip.creatis.insa-ly

Check the output files

In [28]:
! tree {VipSession("demo-single-job", verbose=False).output_dir}

vip_outputs/demo-single-job
├── 13-06-2023_20:08:05
│   └── signal.mrui--parameters.txt.tgz
│       ├── parameters.txt
│       ├── signal_Correlation.txt
│       ├── signal.mrui
│       ├── signal_quest2.txt
│       ├── signal_quest_back.mrui
│       ├── signal_quest_estim.mrui
│       └── stdout_cquest.txt
├── 13-06-2023_20:13:13
│   └── signal.mrui--parameters.txt.tgz
│       ├── parameters.txt
│       ├── signal_Correlation.txt
│       ├── signal.mrui
│       ├── signal_quest2.txt
│       ├── signal_quest_back.mrui
│       ├── signal_quest_estim.mrui
│       └── stdout_cquest.txt
└── session_data.json

4 directories, 15 files


# Remove Temporary Data from VIP

*After the download, **your input and output data are still on VIP** (https://vip.creatis.insa-lyon.fr/)*

In [29]:
my_session.vip_input_dir, my_session.vip_output_dir

('/vip/Home/API/demo-single-job/INPUTS',
 '/vip/Home/API/demo-single-job/OUTPUTS')

*Please remove your temporary data when a session is over.*

In [30]:
VipSession("demo-single-job").finish()
VipSession("demo-parallel-jobs").finish()


=== SESSION 'demo-single-job' ===

Output directory: vip_outputs/demo-single-job
<< Session restored from its output directory


=== FINISH ===

Ending Session: demo-single-job
Removing session data
---------------------
[vip] /vip/Home/API/demo-single-job ... Done.
---------------------

Updating workflows status
-------------------------
workflow-sPobH7: Removed
workflow-v0T4Ux: Removed
-------------------------
All output data have been removed from VIP.
Session 'demo-single-job' is now over.

>> Session saved


=== SESSION 'demo-parallel-jobs' ===

Output directory: vip_outputs/demo-parallel-jobs
<< Session restored from its output directory


=== FINISH ===

Ending Session: demo-parallel-jobs
Removing session data
---------------------
[vip] /vip/Home/API/demo-parallel-jobs ... Done.
---------------------

Updating workflows status
-------------------------
workflow-wLwvYb: Removed
-------------------------
All output data have been removed from VIP.
Session 'demo-parallel-jobs' 

*The output data downloaded on your computer is yours to remove*

In [31]:
! rm -r vip_outputs